## 18. Transfer Learning

- Transfer Learningdms 이미 대규모 데이터셋에서 학습된 모델(예: ImageNet에서 학습된 CNN 모델)을 가져와, **새로운 과제에 맞게 일부 또는 전체를 재학습** 시키는 기법
    - **Feature Extraction (특징 추출)**: 기존 모델의 convolution layer는 그대로 두고, 마지막 분류기(classifier)만 새 데이터에 맞게 교체하고 학습.
    - **Fine-tuning (미세조정)**: 기존 모델의 일부 또는 전체 layer의 가중치를 **새로운 데이터에 맞춰 추가로 학습**.
- 이 방법은 **학습에 필요한 데이터가 적거나, 학습 시간을 줄이고자 할 때 효과적**
- 전이 학습의 성능은 **새로운 데이터셋이 원래 모델이 학습한 데이터와 얼마나 유사한지**, 그리고 **사용 가능한 데이터의 양**에 따라 달라지기에 따라서 **적절한 사전 학습 모델을 선택하고, 얼마나 많은 layer를 고정시킬지(freeze) 또는 학습시킬지 결정하는 것이 중요**

In [3]:
import torch

# CPU/GPU 선택
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [11]:
import torch
from torchvision import datasets, transforms

# 이미지 변환(전처리)
# 모델에 맞게 224 * 224로 변환
transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406] , std=[0.229, 0.224, 0.225])
])

# CIFAR-10은 10개의 클래스에 걸쳐 총 60,000개의 32x32 컬러 이미지로 구성된 데이터셋
train_dataset = datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=8,
)

test_dataset = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=8,
)

print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(test_dataset))

Train dataset size: 50000
Validation dataset size: 10000


In [13]:
import torch.nn as nn
from torchvision import models

# 사전 학습된(pretrained) ResNet-18 모델
# weights=ResNet18_Weights.DEFAULT는 ImageNet으로 학습된 가중치를 사용
weights = models.ResNet18_Weights.DEFAULT
model = models.resnet18(weights=weights)

# 원하는 class의 개수로 output을 변경
model.fc = nn.Linear(512, 10)
print(model.fc)

Linear(in_features=512, out_features=10, bias=True)


In [7]:
import torch.optim as optim

model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
  model.train()
  for batch_idx, (data, label) in enumerate(train_loader):
    data, label = data.to(device), label.to(device)
    optimizer.zero_grad()
    output = model(data)
    train_loss = loss_fn(output, label)
    train_loss.backward()
    optimizer.step()

  model.eval()
  val_loss = 0
  correct = 0
  with torch.no_grad():
    for data, label in test_loader:
      data, label = data.to(device), label.to(device)
      output = model(data)
      val_loss += loss_fn(output, label).item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(label.view_as(pred)).sum().item()

  val_loss /= len(test_loader.dataset)
  accuracy = 100. * correct / len(test_loader.dataset)

  print(f'Epoch {epoch+1}, Loss: {train_loss.item():.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1, Loss: 0.3846, Val Loss: 0.0153, Accuracy: 83.37%
Epoch 2, Loss: 0.3918, Val Loss: 0.0145, Accuracy: 84.34%
Epoch 3, Loss: 1.3060, Val Loss: 0.0115, Accuracy: 87.81%
Epoch 4, Loss: 0.1076, Val Loss: 0.0108, Accuracy: 88.52%
Epoch 5, Loss: 0.2412, Val Loss: 0.0126, Accuracy: 88.43%
Epoch 6, Loss: 0.0039, Val Loss: 0.0134, Accuracy: 88.16%
Epoch 7, Loss: 0.0357, Val Loss: 0.0126, Accuracy: 88.92%
Epoch 8, Loss: 0.0249, Val Loss: 0.0137, Accuracy: 88.59%
Epoch 9, Loss: 0.0082, Val Loss: 0.0124, Accuracy: 89.83%
Epoch 10, Loss: 0.1971, Val Loss: 0.0141, Accuracy: 88.91%
